<a href="https://colab.research.google.com/github/holoho/CP/blob/main/CP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests, json, os, shutil, time, glob
from bs4 import BeautifulSoup
import pandas as pd
import tensorflow as tf

In [ ]:
# 만개의 레시피 이미지 크롤링
from urllib.request import urlretrieve # 이미지를 저장할 때 사용
 
food_list = ['김치찌개','계란볶음밥']
 
start = time.time()  
for name in food_list:# 반복문을 사용해 음식 종류 list안에 있는 음식별로 크롤링 할 수 있도록 설정
    
    base_url = 'https://www.10000recipe.com/recipe/list.html?q={}&order=reco&page={}' 
    n = 1
    
    for page in range(1, 5):  
        try:
            url = base_url.format(name, page)
            response = requests.post(url)
        
            soup = BeautifulSoup(response.content, 'html.parser')
            images = soup.findAll("a", {"class" : "common_sp_link"})
        
            if not images: # 빈 리스트인 경우
                raise ex
                
            imgurl = [] # 이미지 url을 저장하기 위한 변수
 
            for i in images:
                temp = i.img['src'] 
                
                if temp.endswith('png') is False: # png형식의 파일은 저장하지 않는다. 
                    imgurl.append(i.img['src']) # 이미지의 source 주소를 가져와 imgurl list에 추가한다
            
            for i in imgurl:
                urlretrieve(i, "/content/drive/MyDrive/AI/projectp/pp/" + name + "-" + str(n) + ".jpg") # 크롤링한 사진에 1씩 값을 증가해 저장한다
                n += 1
            
        except Exception as ex: # 이미지가 없는 경우 종료
            print(name + " 크롤링 완료")
            print(ex)
            break
            
print("총 소요시간 : " + str(time.time() - start))  


In [ ]:
image_directory = '/content/drive/MyDrive/AI/projectp/pp'
df = pd.read_excel('/content/drive/MyDrive/AI/projectp/통합 식품영양성분DB_음식_20221110.xlsx', header = 3) 

In [ ]:
df = df[['식품명','1회제공량','에너지(㎉)','수분(g)','단백질(g)','지방(g)','탄수화물(g)','총당류(g)','총 식이섬유(g)']]

In [ ]:
# 이미지 파일 있는 식품명만 선택
df1 = df.loc[df['식품명'].isin(['칼국수','해장국','햄버거','김밥','김치찌개','냉면','계란볶음밥','닭가슴살샐러드','돈가스','떡국','샌드위치','우동','연어샐러드','카레',
                       '짜장면','짬뽕','제육볶음덮밥','순댓국','비빔밥','불고기덮밥','뼈해장국','감자탕','갈비탕','닭갈비','순두부찌개','콩나물국밥','달걀말이','족발',
                       '보쌈','백반','라면','떡볶이'])] 

In [ ]:
df2 = df1.reset_index(drop=True) # 인덱스 컬럼 제거

In [ ]:
df2

In [ ]:
data = df2.drop(labels=[0,4,5],axis=0) # 식품명 중복 제거

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data

In [ ]:
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
image_exts = ['jpeg','jpg']
for image_class in os.listdir(image_directory): 
    for image in os.listdir(os.path.join(image_directory, image_class)):
        image_path = os.path.join(image_directory, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))

In [ ]:
data1 = tf.keras.utils.image_dataset_from_directory(image_directory)

In [ ]:
data_iterator = data1.as_numpy_iterator()
batch = data_iterator.next()

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# 이미지 numpy array 변형
batch[0].shape

In [ ]:
# class가 33개이기에 전부 binary로 변경하기 위해 시도(안해도 모델 돌아감)
#from sklearn.preprocessing import MultiLabelBinarizer
#mlb = MultiLabelBinarizer()
#yh = mlb.fit_transform(['칼국수','해장국','햄버거','김밥','김치찌개','냉면','계란볶음밥','닭가슴살샐러드','돈가스','떡국','샌드위치','우동','연어샐러드','카레',
#                       '짜장면','짬뽕','제육볶음덮밥','순댓국','비빔밥','불고기덮밥','뼈해장국','감자탕','갈비탕','닭갈비','순두부찌개','콩나물국밥','달걀말이','족발',
#                       '보쌈','백반','라면','떡볶이'])

In [ ]:
# 데이터 scaling
data1 = data1.map(lambda x,y: (x/255, y))
scaled = data1.as_numpy_iterator()

In [ ]:
train_size = int(len(data1)*.7)
val_size = int(len(data1)*.2)+1
test_size = int(len(data1)*.1)+1

In [ ]:
train = data1.take(train_size)
val = data1.skip(train_size).take(val_size)
test = data1.skip(train_size+val_size).take(test_size)

In [ ]:
len(train)

In [ ]:
import itertools
import keras
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import img_to_array, load_img 
from keras.models import Sequential, Model 
from keras.layers import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense  
from keras import applications  
from keras.utils.np_utils import to_categorical  
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math  
import datetime
import time

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
hist.history

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

In [ ]:
img = cv2.imread('냉면-10.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))
yhat

In [ ]:
DIRECTORY = '/content/drive/MyDrive/AI/projectp/pp'

CATEGORIES = ['칼국수','해장국','햄버거','김밥','김치찌개','냉면','계란볶음밥','닭가슴살샐러드','돈가스','떡국','샌드위치','우동','연어샐러드','카레',
                       '짜장면','짬뽕','제육볶음덮밥','순댓국','비빔밥','불고기덮밥','뼈해장국','감자탕','갈비탕','닭갈비','순두부찌개','콩나물국밥','달걀말이']